<a href="https://colab.research.google.com/github/caleb-stewart/A-Z_Machine-Learning/blob/main/convolutional_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network

### Importing the libraries

In [1]:
# !pip install --upgrade tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
tf.__version__

'2.18.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
# Apply transformation on the training set
# This avoids overfitting
# We use Image Augmentation

# Dividing by 255 will get pixel values between 0 and 1
train_datagen = ImageDataGenerator(rescale=1.0/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

# This is a binary outcome (cat or dog)
training_set = train_datagen.flow_from_directory('dataset/training_set', target_size=(64, 64), batch_size=32, class_mode='binary')

### Preprocessing the Test set

In [ ]:
# This is for the test set
test_datagen = ImageDataGenerator(rescale=1.0/255)
# Different directory for the test_set
test_set = test_datagen.flow_from_directory('dataset/test_set', target_size=(64, 64), batch_size=32, class_mode='binary')

## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
# filters is how many feature detectors do we want. 32 is a classic architecture
# kernel is the size of the feature detector
# input shape will be what we resized it to (64, 64, 3). 3 because of rgb

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [ ]:
# Apply max pooling

# pool size is the size of teh area to search for the max pixel number
# stride is how many pixels the pool size moves
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
# Adds a second convolutional layer with max pooling
# No input_shape this time because we only do that on the first layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))


### Step 3 - Flattening

In [ ]:
# Apply flattening. Makes it a large vector
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
# Add a new fully connected layer

# units is the number of hidden neurons
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
# We are doing binary classification. So either 1 or 0
# sigmoid because this is the output layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
# Connect the cnn to an optimizer
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
# train the cnn on the training set
# validation_set is the test set
# Epochs is how many times it runs
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

## Part 4 - Making a single prediction

In [ ]:
from tensorflow.keras.preprocessing import image

# Load a single image. We also need to resize the images to what we trained it on
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64, 64))
# Convert test_image into an array
test_image = image.img_to_array(test_image)
# Make it in a format expected by the predict method
test_image = np.expand_dims(test_image, axis=0)

result = cnn.predict(test_image)
# Show which animal corresponds to 1 and 0
training_set.class_indices

if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

print(prediction)